In [42]:
import requests
import os
from bs4 import BeautifulSoup
import hashlib
import pandas as pd

In [115]:
pathSM = "data/sitemap.xml"
if os.path.isfile(pathSM):
    print(pathSM, ": already done")
else:
    url = 'https://www.mottmac.com/sitemap.xml'
    r = requests.get(url, allow_redirects=True)
    open(pathSM, 'wb').write(r.content)

pathRobot = "data/robots.txt"
if os.path.isfile(pathRobot):
    print(pathRobot, ": already done")
else:
    url = 'https://www.mottmac.com/robots.txt'
    r = requests.get(url, allow_redirects=True)
    open(pathRobot, 'wb').write(r.content)

data/sitemap.xml : already done
data/robots.txt : already done


In [116]:

with open(pathSM, 'r') as f:
    data = f.read()
pages = BeautifulSoup(data, "xml")

In [117]:
urlList = []
URLs = pages.find_all('url')
for url in URLs:
    ref = url.find_all('xhtml:link', {'rel':'alternate'})[0]
    urlList.append(ref.get('href'))
urlList = list(set(urlList)) 
urlList = [x for x in urlList if not "/job" in x ]
urlList = [x for x in urlList if not "/download/" in x ]
urlList

['https://www.mottmac.com/releases/mott-macdonald-appointed-digital-and-innovation-healthcare-advisor-by-the-leeds-teaching-hospitals-nhs-trust-ltht',
 'https://www.mottmac.com/releases/mott-macdonald-appoints-bruno-trouille-as-senior-hydropower-advisor',
 'https://www.mottmac.com/releases/speyside-biomass-power-station-receives-74-million-funding-uk',
 'https://www.mottmac.com/careers/giving-up-life-in-the-fast-lane',
 'https://www.mottmac.com/en-US/article/37197/bioengineered-oyster-reef-demonstration',
 'https://www.mottmac.com/careers/corporate-services',
 'https://www.mottmac.com/article/466/balfour-beatty-mott-macdonald-revolutionary-s',
 'https://www.mottmac.com/en-US/article/37399/raritan-river-force-main',
 'https://www.mottmac.com/releases/mott-macdonald-successfully-provides-environmental-and-social-advice-on-the-largest-solar-pv-complex-in-africa',
 'https://www.mottmac.com/careers/mott-macdonald-listed-on-glassdoor-best-places-to-work-2023',
 'https://www.mottmac.com/en-US

In [118]:
def getTitle(row):
    if not os.path.exists("data/cache/"+row["hash"]):
        return ""
    try:
        with open("data/cache/"+row["hash"]) as fp:
            soup = BeautifulSoup(fp, 'html.parser')
        result = soup.title.text
    except:
        result = ""
    return result

def getKeywords(row):
    if not os.path.exists("data/cache/"+row["hash"]):
        return ""
    try:
        with open("data/cache/"+row["hash"]) as fp:
            soup = BeautifulSoup(fp, 'html.parser')
        result = soup.select_one('meta[name=keywords]')['content']
    except:
        result = ""
    return result

def getDescription(row):
    if not os.path.exists("data/cache/"+row["hash"]):
        return ""
    try:
        with open("data/cache/"+row["hash"]) as fp:
            soup = BeautifulSoup(fp, 'html.parser')
        result = soup.select_one('meta[name=description]')['content']
    except:
        result = ""
    return result

In [119]:
df = dict()
for url in urlList:
    df[url] = hashlib.md5(url.encode()).hexdigest()
df = pd.DataFrame.from_dict(df, orient='index').reset_index()
df.columns = ["url","hash"]
df

,url,hash
0,https://www.mottmac.com/releases/mott-macdonal...,ae00a6f326032789c8f62725015567f9
1,https://www.mottmac.com/releases/mott-macdonal...,d5cadd4b35b047fc62d9bac02fb5e522
2,https://www.mottmac.com/releases/speyside-biom...,8fac9a3e4b3dbdc711bb9cac3b1f21ef
3,https://www.mottmac.com/careers/giving-up-life...,29ebaa0e213b404db68baa57a7610624
4,https://www.mottmac.com/en-US/article/37197/bi...,28ba2edb64e266ce881c1c6e039bf505
...,...,...
4002,https://www.mottmac.com/article/11603/projects...,47e3aeb40050521095a7b3ef774520fb
4003,https://www.mottmac.com/releases/national-coll...,b23b4df9e9fedfe5e5dcb33458a99d7a
4004,https://www.mottmac.com/article/642/mott-macdo...,4755783c81de654818b781a126b59006
4005,https://www.mottmac.com/article/529/mott-macdo...,a879e1ea323939740256a188945aec41


In [120]:
for ix,row in df.iterrows():
    name = row["hash"]
    url = row["url"]

    if not os.path.exists("data/cache/"+name):
        page_sourced = requests.get(url).content 
        html_content = BeautifulSoup(page_sourced, "html.parser")
        with open("data/cache/"+name, 'w') as f:
            f.write(str(html_content))
        print(name,"saved")

8c9791c11a38f57525a4a2d6522be994 saved
6eb9c41b37cce9888bf2d04a17c21963 saved
54fac9347d68718491677bcda43166be saved
10619888ac8803a06c2650074a1c8825 saved
9f8a05eaf8b2b2a070b9ee320fb2d921 saved
904d156b092edd6d90c663225ae5f520 saved
886aaac1c723aa3c5567d5baaf208d91 saved
6a991b5e7d243ab651d59ee08c6a100b saved
ecdc8d0b5155a43520d99fadd95bfcab saved
b6d623797dabc58da481b7985d2f8312 saved
19e26cd6be2419ffc58293a44251e403 saved
9a97e41fe0cef6f82663b661e8a9c8ca saved
842684a536c5e831ae3d1589177679d5 saved
60ff45c48ba0b4a8c6e104c3da3c3db0 saved
375ebce5dd541dbe760d11096bbdf371 saved
008b44aee6e22dd0011c4136caa904d1 saved
522b15f548a2ef561ff8acdeed90ac03 saved
00eb1abe604f812df29ab69a811e6d3c saved
d4b4fb85a713cabd38ff7388d5122bcc saved
e186ce9981c36311b8a7f87f68ab613d saved
58cf504abf38a50abd820f92ac8c58bf saved
cfaa55fad828f0573c4c88cfbdd1c97c saved
e05a662d3331dc8f1ae6d84e87e4da1e saved
fd4dc61771788828e95dac2b39b5ec39 saved
e381ce9e2e1449e9f16f8c5f45caec36 saved
59891b1531f25bd7e1d43b828

In [121]:
if "title" not in list(df.columns):
    df["title"] = df.apply(lambda row:getTitle(row), axis=1)
    df["description"] = df.apply(lambda row:getDescription(row), axis=1)
    df["keywords"] = df.apply(lambda row:getKeywords(row), axis=1)
    df.to_parquet("data/pages.parquet.gzip")
df

,url,hash,title,description,keywords
0,https://www.mottmac.com/releases/mott-macdonal...,ae00a6f326032789c8f62725015567f9,Mott MacDonald appointed digital and innovatio...,Mott MacDonald has been appointed digital and ...,"advisor,innovation,Leeds,LTHT,Mott MacDonald,N..."
1,https://www.mottmac.com/releases/mott-macdonal...,d5cadd4b35b047fc62d9bac02fb5e522,Mott MacDonald appoints Bruno Trouille as seni...,Mott MacDonald has appointed Bruno Trouille as...,"Business development,dams,energy,Hydro,Hydropo..."
2,https://www.mottmac.com/releases/speyside-biom...,8fac9a3e4b3dbdc711bb9cac3b1f21ef,Speyside biomass power station receives £74 mi...,The £74 million Speyside biomass power station...,"biomass,distillery,forestry,Green Investment B..."
3,https://www.mottmac.com/careers/giving-up-life...,29ebaa0e213b404db68baa57a7610624,Giving up life in the fast lane - Mott MacDonald,,
4,https://www.mottmac.com/en-US/article/37197/bi...,28ba2edb64e266ce881c1c6e039bf505,Bioengineered oyster reef demonstration - Mott...,,
...,...,...,...,...,...
4002,https://www.mottmac.com/article/11603/projects...,47e3aeb40050521095a7b3ef774520fb,New York Harbor water siphon replacement - Mot...,"In 2005, Mott MacDonald was retained as a Join...","environment,hydrostatic pressure,maintenance,N..."
4003,https://www.mottmac.com/releases/national-coll...,b23b4df9e9fedfe5e5dcb33458a99d7a,National College for Nuclear officially opens ...,The £7.5 million National College for Nuclear ...,"advisory,Building,college,construction,Consult..."
4004,https://www.mottmac.com/article/642/mott-macdo...,4755783c81de654818b781a126b59006,Mott MacDonald appointed consultant on high sp...,,
4005,https://www.mottmac.com/article/529/mott-macdo...,a879e1ea323939740256a188945aec41,Mott MacDonald opens office in Paris - Mott Ma...,,


# Cleaning the text

In [122]:
import os

import numpy as np
import pandas as pd
import io
from lxml import html
from bs4 import BeautifulSoup
import trafilatura

In [123]:
files = os.listdir('data/cache/')
file_names = []
for name in files:
    if not ('type' in name):
        file_names.append(name)

In [124]:
articles = []
errors = []
# Takes around 16s for 456 articles
# 2Min19 for 3000
for file_name in file_names:
    with io.open(f'data/cache/{file_name}', mode="r", encoding="utf-8") as f:
        try:
            mytree = html.fromstring("".join(f.readlines()))
        except Exception as e:
            print(e)
            errors.append(file_name)
            continue
        try:
            content = trafilatura.extract(mytree)
            articles.append((file_name, content))
        except Exception as e:
            print(e)
            errors.append(file_name)

Document is empty


In [125]:
len(dfContent)

456

In [126]:
import re
from typing import Set
from transformers import GPT2TokenizerFast

import numpy as np
from nltk.tokenize import sent_tokenize

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    """count the number of tokens in a string"""
    return len(tokenizer.encode(text))

def reduce_long(
    long_text: str, long_text_tokens: bool = False, max_len: int = 590
) -> str:
    """
    Reduce a long text to a maximum of `max_len` tokens by potentially cutting at a sentence end
    """
    if not long_text_tokens:
        long_text_tokens = count_tokens(long_text)
    if long_text_tokens > max_len:
        sentences = sent_tokenize(long_text.replace("\n", " "))
        ntokens = 0
        for i, sentence in enumerate(sentences):
            ntokens += 1 + count_tokens(sentence)
            if ntokens > max_len:
                return ". ".join(sentences[:i][:-1]) + "."

    return long_text

In [127]:
dfContent = pd.DataFrame(articles, columns = ['hash', 'content'])

In [130]:
dfContent["content"] = dfContent["content"].apply(lambda x: reduce_long(x, 750))
dfContent["tokens"] = dfContent["content"].apply(lambda x: count_tokens(x))

In [131]:
dfContent.to_parquet("data/content.parquet.gzip")
dfContent

,hash,content,tokens
0,a8414fff120d9c5f9073376b27f52dcf,"Mott MacDonald divisional director, Anne Kerr,...",111
1,3b1ca0a60662c15c1cd6e6fa0206f4a0,A consortium including Cheung Kong Infrastruct...,341
2,fb558366bffeb1834d6fea64b41236ba,"Mott MacDonald, the global engineering, manage...",503
3,b0b1a2444987ebf09f8fe22a170cc6dd,We were proud to be platinum sponsor of The Ec...,584
4,e8f484cc2df87052b1617e79b3dee16a,Mott MacDonald offers a wide variety of traffi...,98
...,...,...,...
4001,24438270208be5df0f6c89e56b2b7926,"Mott MacDonald, assisted by Rand Europe, has b...",416
4002,7f41c6131fb54bc4522d9f760dee5c34,"Phil Vigor, principal airport planner The stea...",523
4003,d81b9d188f526726d84381fa7b041bff,Additional growth potential has been identifie...,140
4004,8397cf71f1d5a56f2815b41bf2a61088,Mott MacDonald and ADP have been appointed by ...,559
